In [6]:
from SpiderV2 import Spider
from IndexV2 import Database, Index, RawInfoIndex
from Tokenize import Tokenize
from Searcher import Search
import time

In [7]:
db = Database(SearchEngine="ForSpiderTest")
index = Index()
raw_storage = RawInfoIndex()
tokenizer = Tokenize()
raw_storage.read_from_database(db)
index.read_fw_index_from_database(db)
index.read_ivi_index_from_database(db)

In [1]:

url = "https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki"
#"https://en.wikipedia.org/wiki/Climate_change", "https://riskofrain.fandom.com/wiki/Main_Page", "https://www.thairath.co.th/news/local/bangkok/2635906" 
Spider.set_base_domains(["https://www.thairath.co.th/", "https://riskofrain.fandom.com/wiki/Main_Page", "https://en.wikipedia.org/", "https://iot-kmutnb.github.io/", "https://riskofrain2.fandom.com/"])
Spider.max_depth = 0

spider = Spider(url)
success = spider.generate_next_soup()

raw_storage.read_from_database(db)

if success:
    info = spider.crawl({})
    raw_storage.modify_index(Spider.queue[Spider.queue_front-1], info[0], info[1], info[2])
    raw_storage.save_to_database(db)
else:
    if Spider.get_base_domain(url) in Spider.all_robots:
        Spider.unaccessible_urls.append(url)
    print("Cannot crawl to the website.")

NameError: name 'Spider' is not defined

In [8]:
for url in raw_storage.index:
    tokens = tokenizer.tokenize(raw_storage.index[url]["text"])
    tokens = tokenizer.filter(tokens)
    tokens = tokenizer.make_counter(tokens)
    index.modify_index(url, tokens)
raw_storage.save_to_database(db)
index.save_fw_index_to_database(db)
index.save_ivi_index_to_database(db)

In [5]:
print(raw_storage.get_ref_count("https://riskofrain.fandom.com/wiki/Main_Page"))

1


In [7]:
raw_storage.remove_urls(["https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki"])
index.remove_urls(raw_storage.url_to_be_deleted)

In [10]:
print(index.ivi_index["risk"])

{'https://en.wikipedia.org/wiki/Climate_change': 39, 'https://riskofrain.fandom.com/wiki/Main_Page': 26, 'https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki': 18}


In [9]:
print(Search().search(["ไฟฟ้า","โหนด","ตัวเก็บประจุ"], raw_storage.index, index.fw_index, index.ivi_index))

[('https://iot-kmutnb.github.io/blogs/electronics/mesh_and_nodal_analysis', 0.139285167282785, 0, {}), ('https://iot-kmutnb.github.io/blogs/electronics/r2r_lab', 0.12397450238601378, 0, {}), ('https://iot-kmutnb.github.io/blogs/electronics/rlc_circuits', 0.08967580001152524, 0, {}), ('https://iot-kmutnb.github.io/blogs/electronics/dc_circuit_analysis', 0.02035772553328785, 0, {}), ('https://iot-kmutnb.github.io/blogs/electronics/dc_sources_analysis', 0.019197889129605634, 0, {}), ('https://iot-kmutnb.github.io/blogs/electronics/oscilloscopes', 0.014593870469564466, 0, {}), ('https://iot-kmutnb.github.io/blogs/electronics/inductance_measurement', 0.012328919624517381, 0, {}), ('https://iot-kmutnb.github.io/blogs/tools/pulseview_rigol_ds1054z', 0.007864533952142762, 0, {}), ('https://iot-kmutnb.github.io/blogs/electronics/pulse_gen_logic_gates', 0.006904194989729734, 0, {}), ('https://iot-kmutnb.github.io/blogs', 0.006615598446037285, 0, {}), ('https://iot-kmutnb.github.io/blogs/electron

In [ ]:
#bruh